In [ ]:
import pandas as pd
topics_dates = pd.read_csv('Data/13T Gensim/topics_dates.csv')

In [ ]:
data=pd.read_csv('Data/no_punc_tokes.csv', index_col='Unnamed: 0')

In [3]:
topics_dates.head()

,Unnamed: 0.1,Unnamed: 0,topics,date,subjects,misinfo,misinfo_gen,rts,likes,replies,quotes
0,0,0,7,2020-03-01,"truth, republicans, conspiracy, 92",neutral,neutral,0.0,1.0,1,0.0
1,1,1,7,2020-03-01,"truth, republicans, conspiracy, 92",neutral,neutral,0.0,0.0,0,0.0
2,2,2,99,2020-03-01,not in model,not in model,not in model,0.0,8.0,0,0.0
3,3,3,11,2020-03-02,"misinformation, fda, anti-vaxxers",high anti,anti,1.0,4.0,0,0.0
4,4,4,3,2020-03-03,"covid studies, clinical trials, evidence",high anti,anti,0.0,0.0,1,0.0


In [4]:
data.head()

,datetime,lang,rts,likes,quotes,replies,hr,date,tokes,no_punc
0,2020-03-01T20:43:33.000Z,en,0.0,1.0,0.0,1,20.0,2020-03-01,"['@ronnie_haydon', 'almost', 'certainly', 'man...","['@ronnie_haydon', 'almost', 'certainly', 'man..."
1,2020-03-01T17:03:47.000Z,en,0.0,0.0,0.0,0,17.0,2020-03-01,"['[', 'ivermectin', 'paste', 'dewormer', '-', ...","['[', 'ivermectin', 'paste', 'dewormer', '-', ..."
2,2020-03-01T12:15:08.000Z,en,0.0,8.0,0.0,0,12.0,2020-03-01,"['🏁', 'the', 'bohemia', 'team', 'is', 'gearing...","['🏁', 'the', 'bohemia', 'team', 'is', 'gearing..."
3,2020-03-02T15:25:11.000Z,en,1.0,4.0,0.0,0,15.0,2020-03-02,"['.', '@ginagh', 'works', 'as', '#isglobal', '...","['@ginagh', 'works', 'as', '#isglobal', 'direc..."
4,2020-03-03T21:00:33.000Z,en,0.0,0.0,0.0,1,21.0,2020-03-03,"['@liezjkc', 'categorically', 'untrue', '.', '...","['@liezjkc', 'categorically', 'untrue', 'warre..."


In [3]:
topics_dates['rts'] = data.rts
topics_dates['likes'] = data.likes
topics_dates['replies'] = data.replies
topics_dates['quotes'] = data.quotes

In [44]:
topics_dates['amp'] = topics_dates.rts + topics_dates.quotes

In [26]:
topics_dates.to_csv('Data/13T Gensim/topics_dates.csv')

In [4]:
topic = {
    0: 'horse paste/dewormer, does/not work, pharma',
    1: 'getting covid, ?how to take',
    2: 'social influencer',
    3: 'clinical studies',
    4: 'vaccinations',
    5: 'money sign, international use',
    6: 'horse vs human use, rofl emoji',
    7: '92, truth, republicans',
    8: 'alternative treatments',
    9: 'social influencers',
    10: 'nobel prize',
    11: 'misinformation, fda, anti-vaxxers',
    12: 'uttar pradesh',
    99: 'not in model'
}

Topics 2, 3, 4, 8, 9, 10, 12 are the more coherent ones, so I elected to only plot those

In [5]:
topics_dates['subjects'] = topics_dates.topics.apply(lambda x: topic[x])

In [20]:
top_plot=topics_dates[topics_dates.topics.isin([2, 3, 4, 8, 9, 10, 12])].groupby(['date', 'topics']).count()

In [21]:
top_plot=top_plot.reset_index()

In [22]:
top_plot['subjects']=top_plot.topics.apply(lambda x: topic[x])

In [19]:
top_piv=topics_dates[topics_dates.topics.isin([2, 3, 4, 8, 9, 10, 12])].pivot_table(index='subjects', columns='date', aggfunc='count', values='Unnamed: 0', fill_value=0)

In [29]:
list(set(top_plot.subjects))

['nobel prize',
 'uttar pradesh',
 'alternative treatments',
 'social influencer',
 'vaccinations',
 'clinical studies',
 'social influencers']

Using Plotly to generate ridgeline plot of normalized topic volumes over time.

Borrows heavily from: https://www.python-graph-gallery.com/ridgeline-graph-plotly

In [95]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

color_discrete_map={
    'nobel prize':'darkcyan',
    'uttar pradesh':'lightcoral',
    'alternative treatments':'lightslategrey',
    'social influencer':'darkgoldenrod',
    'vaccinations':'lightgreen',
    'clinical studies':'lightpink',
    'social influencers':'lightseagreen'}

# the idea behind this ridgeline plot with Plotly is to add traces manually, each trace corresponding to a particular year's temperature distribution
# thus, we are to store each year's data (temperatures and their respective count) in seperate arrays or pd.series that we store in a dictionnary to retrieve them easily
array_dict = {} # instantiating an empty dictionnary
for topics in list(set(top_plot.subjects)):
    array_dict[f'x_{topics}'] = top_plot[top_plot['subjects']==topics].date # storing the date for each topic
    array_dict[f'y_{topics}'] = top_plot[top_plot['subjects']==topics]['Unnamed: 0'] # storing the count for each date
    array_dict[f'y_{topics}'] = (array_dict[f'y_{topics}'] - array_dict[f'y_{topics}'].min()) \
                                / (array_dict[f'y_{topics}'].max() - array_dict[f'y_{topics}'].min()) # we normalize the array (min max normalization)
    array_dict[f'c_{topics}'] = color_discrete_map[topics]                                

# once all of this is done, we can create a plotly.graph_objects.Figure and add traces with fig.add_trace() method
# since we have stored the temperatures and their respective count for each year, we can plot scatterplots (go.Scatter)
# we thus iterate over year_list and create a 'blank line' that is placed at y = index, then the corresponding temperature count line
fig = go.Figure()
for index, topics in enumerate(list(set(top_plot.subjects))):                        
    fig.add_trace(go.Scatter(
                            x=[pd.to_datetime('2020/03/01'), pd.to_datetime('2022/10/13')], y=np.full(2, len(list(set(top_plot.subjects)))-index),
                            mode='lines',
                            line_color='white'))
    fig.add_trace(go.Scatter(
                            x=array_dict[f'x_{topics}'],
                            y=array_dict[f'y_{topics}'] + (len(list(set(top_plot.subjects)))-index) + 0.4,
                            fill='tonexty',
                            name=f'{topics}',
                            line=dict(color=array_dict[f'c_{topics}'])
                            #fillcolor=array_dict[f'c_{topics}']
                            ))   

    
    #plotly.graph_objects' way of adding text to a figure
    fig.add_annotation(
                        x='2020/01/01',
                        y=len(list(set(top_plot.subjects)))-index+0.4,
                        text=f'{topics}',
                        showarrow=False,
                        yshift=10)

# here you can modify the figure and the legend titles
    fig.update_layout(
                title=dict(text='Normalized Ivermectin Topic Volumes through the Pandemic', x=0.5, xanchor='center'),
                showlegend=False,
                xaxis=dict(title='Date'),
                yaxis=dict(showticklabels=False),
                width=2000,
                height=500
                )

fig.show()

In [26]:
top_plot=topics_dates[topics_dates.topics != 99].groupby(['date', 'topics']).count()['Unnamed: 0']

Plotting all topics over time

In [81]:
top_plot=topics_dates[topics_dates.topics.isin([3,8])].groupby(['date', 'subjects']).count()
top_plot.index.get_level_values('date')
import plotly.express as px
fig = px.line(topics_dates[topics_dates.topics.isin([3,8])].groupby(['date', 'subjects']).count(), x = top_plot.index.get_level_values('date'), y='Unnamed: 0', color=top_plot.index.get_level_values('subjects'), width=7500, height=800)
fig.show()